In [ ]:
import DeepFMKit.core as dfm
from DeepFMKit.workers import calculate_ambiguity_boundary_point
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from tqdm import tqdm
import multiprocessing
import os

def generate_and_plot_ambiguity_boundary(
    df_range=np.linspace(1e9, 100e9, 100),    # 1 to 20 GHz
    dm_range=np.logspace(-5, -2, 100),       # 1e-5 to 0.1 rad
    wavelength_nm=1550,
    n_cores=None
):
    """
    Generates and plots the Ambiguity Resolution Boundary landscape.
    """
    if n_cores is None:
        n_cores = os.cpu_count()
        
    f0 = dfm.sc.c / (wavelength_nm * 1e-9)
        
    print("=" * 60)
    print("Generating Ambiguity Resolution Boundary Plot")
    print(f"Carrier Frequency (f0): {f0 / 1e12:.1f} THz")
    print("=" * 60)
    
    jobs = []
    for j, df in enumerate(df_range):
        for i, dm in enumerate(dm_range):
            jobs.append({
                'delta_f': df,
                'delta_m': dm,
                'f0': f0,
                'grid_i': i, 'grid_j': j
            })

    grid_shape = (len(dm_range), len(df_range))
    error_grid = np.zeros(grid_shape)

    if __name__ == "__main__":
        with multiprocessing.Pool(processes=n_cores) as pool:
            results_iterator = pool.imap(calculate_ambiguity_boundary_point, jobs)
            for result in tqdm(results_iterator, total=len(jobs), desc="Calculating Boundary Grid"):
                i, j, error = result
                error_grid[i, j] = error
    
    # --- Plotting ---
    fig, ax = plt.subplots(figsize=(12, 9))

    # Use a logarithmic color scale to visualize the wide range of errors
    norm = LogNorm(vmin=np.pi/100, vmax=np.pi*100)
    
    im = ax.pcolormesh(
        df_range / 1e9, dm_range, error_grid,
        norm=norm, cmap='magma_r', shading='auto'
    )
    fig.colorbar(im, ax=ax, label=r'Coarse Phase Error, $|\delta\Phi_{\rm coarse}|$ (rad)')

    # The critical contour at pi
    CS = ax.contour(
        df_range / 1e9, dm_range, error_grid,
        levels=[np.pi], colors='cyan', linewidths=3
    )
    ax.clabel(CS, inline=True, fontsize=12, fmt=r'$|\delta\Phi_{\rm coarse}| = \pi$')

    ax.set_yscale('log')
    ax.set_xlabel(r'Modulation Amplitude, $\Delta f$ (GHz)', fontsize=14)
    ax.set_ylabel(r"Total Error on m, $|\delta m|$ (rad)", fontsize=14)
    ax.set_title('Ambiguity Resolution Boundary', fontsize=16)
    
    # Add text annotations to show the safe vs. failure regions
    ax.text(0.3, 0.4, 'Ambiguity\nResolved', transform=ax.transAxes,
            ha='center', fontsize=15, color='white', weight='bold')
    ax.text(0.7, 0.8, 'Fringe\nSlip', transform=ax.transAxes,
            ha='center', fontsize=15, color='white', weight='bold')

    ax.grid(True, which='both', linestyle=':', alpha=0.5)
    plt.show()

# --- Run the analysis ---
if __name__ == "__main__":
    generate_and_plot_ambiguity_boundary()